Content
* 0. introduction
* 1. sub-pipeline design: from simulator output (a spread sheet) to payments created
* 2. details of the sub-pipeline:
    * 2.1 load the spread sheet
    * 2.2 create orders (with line-items ids)
    * 2.3 pay the orders
    * 2.4 relavant module (list orders, list payments, list items)
* 3. Reflection and Discussion, what's next

# 0. Introduction
The Orders API integrate parts of the Square ecosystem to track and manage the lifecycle of a purchase. The API can record purchase items, calculate totals, confirm payment, track an order's progress through fulfillment, and update a catalog inventory.

Orders API sits in between catalog api and payments api in our pipeline. Historically, orders api and payments api together function as a transaction API (depreciated). So we will look closely at orders api and payments api.

The Payments API provides the CreatePayment endpoint to take a payment. In this project we are implementing itemized transaction. In other words, we can not only track transaction amount but also the items involved in each transaction. This means we will need the order_id to for the CreatePayment endpoint.

API and endpoint list in this notebook
* locations.list_locations()
* orders.create_order()
* payments.create_payment()

* catalog.list_catalog()
* payments.list_payments()

## Outcome
At the end, we will provide a framework to wrap createOrder and createPayment in one function.

In [1]:
import pandas as pd

In [2]:
## Add this to the first block in your note book to display json
import uuid
from IPython.core.display import display, HTML

import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())
        # This line is missed out in most of the versions of this script across the web, it is essential for this to work interleaved with print statements
#         self._ipython_display_()
        
    def _ipython_display_(self):
        display(HTML('<div id="{}" style="height: auto; width:100%;"></div>'.format(self.uuid)))
        display(HTML("""<script>
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_show_to_level(1)
          document.getElementById('%s').appendChild(renderjson(%s))
        });</script>
        """ % (self.uuid, self.json_str)))

# Since this is copy-pasted wrongly(mostly) at a lot of places across the web, i'm putting the fixed, updated version here, mainly for self-reference


## To use this function, call this, this now works even when you have a print statement before or after the RenderJSON call
# RenderJSON(your_json)
# RenderJSON(json.loads(result.text))

# 1. Pipeline design
The pipeline:
create Items -> create Orders -> create Payments
# 2. about the pipeline
* 2.0 recap: the upstream: Items
* 2.1 load the spread sheet (from simulator output)
* 2.2 create orders (with line-items ids)
* 2.3 pay the orders
* 2.4 relavant module

In [3]:
# module connection
# sandbox connection to "Square SandBox no2"

from square.client import Client
sandbox_token = "EAAAELeY2Gk_kgDuX7SFsmt-d3UzMTsdB9nRf2ew8EjO1wDdnpI1LwOV0KobTx4g"

client = Client(
    access_token = sandbox_token,
    environment='sandbox',
)

## 2.0 recap: retrieve items info

In [4]:
# retrieve item and variation from API
result_variation = client.catalog.list_catalog(types="ITEM_VARIATION")
result_item = client.catalog.list_catalog(types="ITEM")

df_left = pd.json_normalize(result_variation.body.get("objects"))[["id", 
                                               "item_variation_data.name", 
                                               "item_variation_data.price_money.amount",
                                               "item_variation_data.item_id"]]
df_right = pd.json_normalize(result_item.body.get("objects"))[["id", "item_data.name"]]

pd.merge(df_left,
         df_right,
         left_on="item_variation_data.item_id", right_on="id",
         how="left").drop("id_y", axis=1)


,id_x,item_variation_data.name,item_variation_data.price_money.amount,item_variation_data.item_id,item_data.name
0,FJAR6IET4UZB6UUPZJCIV4GU,Regular,NaN,2S33CD4247XXEORTAT5XYOQB,Hat
1,S5UX6NLNGU6RM6HEG2BPGH6D,Regular,NaN,THBCUS7E6THY4UVPKZJU7VSJ,Scarf
2,UEVMQVNCUMKX5TL7KMA6FFVU,Regular,NaN,JQKHXVTRQQUJC4OWF5F6HQFK,Shirt
3,3FG3W4BFXUWML6UFUSFZMN7N,Regular,NaN,Z63NWS6BEBKNPDSR6AXPUSYV,Sunglasses
4,J3VQOGJD4WETWRZIGJV4Z6P7,Small red,2500.0,JQKHXVTRQQUJC4OWF5F6HQFK,Shirt
5,724F4WGRJEXUTK6WUC2SKM6Z,Small blue,2500.0,JQKHXVTRQQUJC4OWF5F6HQFK,Shirt
6,METCIVRJ3HTZ5QQUU2YQTSTJ,Large red,3000.0,JQKHXVTRQQUJC4OWF5F6HQFK,Shirt
7,W3G4W5LRBJ65LKTMLDM2SIB5,Large blue,3000.0,JQKHXVTRQQUJC4OWF5F6HQFK,Shirt
8,6B7HUOAHA4T37Y4HCNBH3JRK,Blue,1000.0,2S33CD4247XXEORTAT5XYOQB,Hat
9,5Y2WYUW246EEZURQMHX2SKI5,Red,1000.0,2S33CD4247XXEORTAT5XYOQB,Hat


## 2.1 load spreadsheet
The spread sheet provides information in each order.
Note: the spread sheet in this notebook is just for demonstration purpose. The actual "spread-sheet" in production will be created on the fly. (more details, see Simulator design notebook)

In [5]:
# read simulator as input
df_sim = pd.read_csv("/home/hadoop/Desktop/SharedFolder/simulator_output_0606.csv")
df_sim

,timestamp_input,timestamp_execute,FJAR6IET4UZB6UUPZJCIV4GU,S5UX6NLNGU6RM6HEG2BPGH6D,UEVMQVNCUMKX5TL7KMA6FFVU,3FG3W4BFXUWML6UFUSFZMN7N,J3VQOGJD4WETWRZIGJV4Z6P7,724F4WGRJEXUTK6WUC2SKM6Z,METCIVRJ3HTZ5QQUU2YQTSTJ,W3G4W5LRBJ65LKTMLDM2SIB5,6B7HUOAHA4T37Y4HCNBH3JRK,5Y2WYUW246EEZURQMHX2SKI5,EMFBRVV54DDDQPVA32SUQUST,WG77ONG6TJ63MCVJSJT5XMFZ,RROX5BQWFMGYR5BV444XRDG6
0,6/6/2021 15:10,6/6/2021 15:14,0,0,0,0,0,0,0,0,0,0,0,4,1
1,6/6/2021 15:20,6/6/2021 15:18,0,0,0,0,0,0,0,0,0,10,5,0,0
2,6/6/2021 15:30,6/6/2021 15:21,0,0,0,0,0,0,0,0,0,0,2,3,2


## explain: 
(without going into details), the simulator-output is an execution plan that we will rely on to create order through Square API.
e.g., at timestamp_execute==6/6/2021 15:14, we we will create order "WG77ONG6TJ63MCVJSJT5XMFZ" x 4 + "RROX5BQWFMGYR5BV444XRDG6", where the tokens are variation id:
* WG77ONG6TJ63MCVJSJT5XMFZ: red scarf
* RROX5BQWFMGYR5BV444XRDG6: sunglasses

(We will describe how to design simulator in details in simulator notebook)

Note: assuming all the orders will come from only one location, then we only need to consider sending one order at each execution timestamp.

## 2.2 create orders (with line-items ids)
We will focus on orders.create_order endpoint in this session. The arguments we will feed in the endpoints are:
* location_id
* line_items: a list consist of item_variations information (e.g., quantity, object_id)

ref: https://developer.squareup.com/docs/orders-api/create-orders


In [6]:
# little detour: about location_id
# api location.list_locations
# ref: https://developer.squareup.com/docs/locations-api

result_location = client.locations.list_locations()
RenderJSON(result_location.body)

# print(result_location.body.get("locations")[0].get("id"))

In [7]:
# construct the line_items formula (again, one row at a time)
# Note: two things:
#  line_itmes need to be pure list (not np.array)
#  the quantity value need to be String (not Int)
# example choose row 0

df_row_to_exe = df_sim.iloc[0, 2:].apply(str)

# df_row_to_exe[df_row_to_exe>0].to_dict()
# df_row_to_exe[df_row_to_exe>0].reset_index()\
# .rename(columns = {'index':'catalog_object_id', 0:'quantity'})
# df_tmp = df_row_to_exe[df_row_to_exe>0].reset_index()\
# .rename(columns = {'index':'catalog_object_id', 0:'quantity'})
# line_items = df_tmp.apply(lambda x: x.to_dict(), axis=1).values

line_items = df_row_to_exe[df_row_to_exe!="0"].reset_index()\
                .rename(columns = {'index':'catalog_object_id', 0:'quantity'})\
                .apply(lambda x: x.to_dict(), axis=1).values
line_items = list(line_items)
line_items

[{'catalog_object_id': 'WG77ONG6TJ63MCVJSJT5XMFZ', 'quantity': '4'},
 {'catalog_object_id': 'RROX5BQWFMGYR5BV444XRDG6', 'quantity': '1'}]

In [13]:
# Update: There is actually a one-line code to achieve this
df_sim.iloc[[0], 2:].to_dict("records")

[{'FJAR6IET4UZB6UUPZJCIV4GU': 0,
  'S5UX6NLNGU6RM6HEG2BPGH6D': 0,
  'UEVMQVNCUMKX5TL7KMA6FFVU': 0,
  '3FG3W4BFXUWML6UFUSFZMN7N': 0,
  'J3VQOGJD4WETWRZIGJV4Z6P7': 0,
  '724F4WGRJEXUTK6WUC2SKM6Z': 0,
  'METCIVRJ3HTZ5QQUU2YQTSTJ': 0,
  'W3G4W5LRBJ65LKTMLDM2SIB5': 0,
  '6B7HUOAHA4T37Y4HCNBH3JRK': 0,
  '5Y2WYUW246EEZURQMHX2SKI5': 0,
  'EMFBRVV54DDDQPVA32SUQUST': 0,
  'WG77ONG6TJ63MCVJSJT5XMFZ': 4,
  'RROX5BQWFMGYR5BV444XRDG6': 1}]

In [69]:
# module create_order
location_id = result_location.body.get("locations")[0].get("id")
line_items

result_create_order = client.orders.create_order(
      body = {
        "order": {
          "location_id": location_id,
#             "customer_id": customer_id,
          "state": "OPEN",
          "line_items": line_items
        },
        "idempotency_key": str(uuid.uuid4())
      }
    )

RenderJSON(result_create_order.body)

In [55]:
# module: search order (based on location_id)
result_search_order = client.orders.search_orders(
  body = {
    "location_ids": [
      location_id
    ]
  }
)

RenderJSON(result_search_order.body)

In [70]:
# retrieve by order_id
# o2QhPTMErWXvoxvwCfvDNuM8Nh4F
# note: payment_id (appear as "tenders"."id": Ni0fotwm56cpIcV8kFMdqA3y8N6YY) is accessible if state: "COMPLETED"

order_id = result_search_order.body.get("orders")[0].get("id")
result = client.orders.retrieve_order(
  order_id = order_id
)
RenderJSON(result.body)

## 2.3 pay the orders
we use payments.create_payment endpoint in this session. The key arguments are:
* amount_money (amount and currency)
* order_id
* source_id

In [77]:
# module pay order in pyments.create_payment entry
# prerequiste: order_id is known (along with the total amount, 
# assocaited with the "order".get("total_money").get("amount") field)
# note: 
# - only works on order that the state is "OPEN"
# - the payment amount needs to match order amount
# - the Orders session in dashboard seems still not working after completed. 
#   my guess is Order Session only deal with order with tags like Delivery, pickup, Shipment.
# - The good news is in Transaction session in dashboard it appear a new transaction. 
#   and we can export orders and transactions in spread-sheet format (In sandbox and production)

# Note: In this one, we need to feed in:
#     source_id: ref: https://developer.squareup.com/docs/testing/test-values#payment-tokens-for-card-payments
#     amount_money: amount, currency
#     order_id
#     idempotency_key (auto generated)



idempotency_key = str(uuid.uuid4())

order_id
pay_amount = result_create_order.body.get("order").get("total_money").get("amount")
pay_currency = result_create_order.body.get("order").get("total_money").get("currency")
result = client.payments.create_payment(
  body = {
    "source_id": "cnon:card-nonce-ok",
    "idempotency_key": idempotency_key,
    "amount_money": {
      "amount": pay_amount,
      "currency": pay_currency
    },
    "order_id": order_id,
  }
)

RenderJSON(result.body)

### Go check the result at dashboard

# Now, all together

In [92]:
###############
# module: wrapper order and pay in one go
def create_order_then_pay_wrapper(line_items_list, location_id, customer_id=None):
#     create_order
    result_create_order = client.orders.create_order(
      body = {
        "order": {
          "location_id": location_id,
#             "customer_id": customer_id,  # In this stage, we will not focus on customer analysis
#           "state": "OPEN", # by default "state": "OPEN"
          "line_items": line_items_list
        },
        "idempotency_key": str(uuid.uuid4())
      }
    )
# if create_order succeed then create_payment (using default source_id: cnon:card-nonce-ok)
    if result_create_order.is_success():    
        order_id = result_create_order.body.get("order").get("id")
        amount = result_create_order.body.get("order").get("total_money").get("amount")
        print("create_order SUCCEED")
    elif result_create_order.is_error():
        print(result_create_order.errors)

    result_create_payment = client.payments.create_payment(
      body = {
        "source_id": "cnon:card-nonce-ok",
        "idempotency_key": str(uuid.uuid4()),
        "amount_money": {
          "amount": amount,
          "currency": "USD"
        },
        "order_id": order_id,
#           "note": note_payment
      }
    )
    if result_create_payment.is_success():
        print("create_payment SUCCEED")
    elif result_create_payment.is_error():
        print(result_create_payment.errors)
        
    return result_create_order, result_create_payment

In [37]:
# create line_items list from dataframe row ( modified with the one-line code)
def line_items_constructor(df_row):
#     df_row_to_exe = df_row.iloc[0, 2:]

#     line_items = df_row_to_exe[df_row_to_exe>0].apply(str).reset_index()
#     line_items.columns = ["catalog_object_id", "quantity"]
#     line_items = line_items.apply(lambda x: x.to_dict(), axis=1).values
#     line_items = list(line_items)
#     return line_items
    return df_row_to_exe[df_row_to_exe>0].dropna(axis=1).to_dict("records")
line_items_constructor(df_sim.loc[[2],:])

ValueError: No axis named 1 for object type Series

In [36]:
df_row_to_exe = df_sim.loc[[2],:].iloc[0, 2:]
pd.DataFrame(df_row_to_exe[df_row_to_exe>0].apply(str)).T

df_row_to_exe = df_sim.loc[[2],:].iloc[:,2:]
(df_row_to_exe[df_row_to_exe>0]).dropna(axis=1).to_dict("records")

[{'EMFBRVV54DDDQPVA32SUQUST': 2,
  'WG77ONG6TJ63MCVJSJT5XMFZ': 3,
  'RROX5BQWFMGYR5BV444XRDG6': 2}]

In [93]:
# test here
# use row 2 to test

location_id 
line_items_list = line_items_constructor(df_sim.loc[[2],:])
result_create_order, result_create_payment = create_order_then_pay_wrapper(line_items_list, location_id)


create_order SUCCEED
create_payment SUCCEED


### adhoc

In [97]:
# how do we know which payments is associated with the order?
# module retrive payment
# note the order_id can be retrived from get_payment
result = client.payments.get_payment(payment_id =result_create_payment.body.get("payment").get("id"))
RenderJSON(result.body)

In [98]:
# list payments (vanilla version)
result = client.payments.list_payments()
RenderJSON(result.body)

# 3. What's Next
We have walked through the API injection workflow (create items, create orders, create payments). At this point the transactions (aka payments) are stored in Square database. Next we will talk about how to retrieve data from Square database using Square API. 
Note: we have already seen the end points (like payments.list_payments) to retrieve(aka list) such data. But we need to retrieve them in a more robust way. We will talk about that in another notebook